# Two Stage VAE

An article from NeurIPS 2018 recently explained a way of altering Variational Autoencoders that leads to better image results. This is measured against the tensorflow implementation of FID, which is used to determine image diversity (i.e. how many different images the generator can produce) and image quality (i.e. how similar the generated images are to real images, as measured by a particular network). 

In brief, the alteration is to change how we measure the reconstruction. A typical measure of the reconstruction is the log probability of the outcome. Given an observation $x$, 


This shortcoming is that the network is more apt to learn the *shape* of the manifold rather than the *distribution of data* on the manifold. Because of this, a standard VAE finds itself producing blurry pictures due to this mismatch in distribution. To remedy this, they propose a simple fix: add a second VAE to learn the distribution of data, once the shape of the manifold has been reasonably learned.

This notebook is an implementation of this idea in pytorch. The network used herein is much smaller, but the result should still be observable with a small network.

In [1]:
import torch as t
import torch.nn as nn
import torchvision.datasets as datasets
import matplotlib.pylab as plt
from torchvision.transforms import Compose, Normalize, ToTensor
from math import sqrt, exp
from numpy import linspace
from time import time

from torchvision.transforms.functional import to_pil_image

In [2]:
batch_size = 125

#Choose data set and batch size
means = [0.49139967861519745, 0.4821584083946076, 0.44653091444546616]
stdevs = [0.2470322324632823, 0.24348512800005553, 0.2615878417279641]
transforms = Compose([ToTensor(), Normalize(means, stdevs)])
training_set = datasets.CIFAR10(root='./data/', train=True, download=False, transform=transforms)
testing_set = datasets.CIFAR10(root='./data/', train=False, download=False, transform=transforms)

training_loader = t.utils.data.DataLoader(dataset=training_set, batch_size=batch_size, shuffle=True, pin_memory=True)
testing_loader = t.utils.data.DataLoader(dataset=testing_set, batch_size=batch_size, shuffle=True, pin_memory=True)

#Set image size. This is done for convenience when changing data.
image_shape = (3, 32, 32)
image_size = image_shape[0]*image_shape[1]*image_shape[2]
out_size = 10

In [3]:
class ShiftDropout(nn.Module):
    def __init__(self, keep_prob=.1, training=True):
        assert keep_prob > 0. and keep_prob < 1.
        super(ShiftDropout, self).__init__()
        self.training = training
        self.keep_prob = keep_prob
        alpha01, lambda01 = 2.90427, 1.07862
        self.min = -alpha01*lambda01*exp(-1)
        
    def forward(self, input):
        #If we are not training, we leave the input as is.
        if not self.training:
            return input
        else:
            mask = t.rand_like(input) < self.keep_prob
            x = mask*input+(~mask)*self.min
            return (mask*input-(1-self.keep_prob)*self.min)/self.keep_prob

def SERLU(input):
    alpha01, lambda01 = 2.90427, 1.07862
    return lambda01*(input*(input>=0)+alpha01*input*t.exp(input)*(input<0))

def init_SNN_weights(module):
    if type(module) is nn.Linear:
        std = 1./sqrt(module.weight.numel())
        nn.init.normal_(module.weight, 0, std)

In [4]:
def print_progress(intro_dict, progress_dict):
    '''
    Prints information from two dictionaries. The first dictionary is printed verbatim,
    while the second dictionary is rounded to 4 decimals. The values of the second
    dictionary must be floats.
    
    :param intro_dict: (dict) Information about the current time. The {key: value}
        pair can have any value that can be used in a str format.
    :param progress_dict: (dict) Information about current progress. The {key: value}
        pair must have a float for the value.
    '''
    intro_text = '{}: {:4}'
    intros = [intro_text.format(key, value) for key, value in intro_dict.items()]
    update_text = '{}: {:.4f}'
    updates = [update_text.format(key, value) for key, value in progress_dict.items()]
    print(*intros, *updates, sep=' - ')

In [ ]:
class MultiBatchNorm(nn.Module):
    def __init__(self, BatchNorm, n_copies=1):
        super(MultiBatchNorm, self).__init__()
        self.main_bn = BatchNorm
        assert type(BatchNorm) in [nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d],
            'Requires BatchNorm to be a torch.nn.BatchNorm object instead of {}'.format(type(BatchNorm))
        #Get proper layer
        if type(BatchNorm) is nn.BatchNorm1d:
            module = nn.BatchNorm1d
        elif type(BatchNorm) is nn.BatchNorm2d:
            module = nn.BatchNorm2d
        elif type(BatchNorm) is nn.BatchNorm3d:
            module = nn.BatchNorm3d
            
        #Collect parameters
        params = [
            BatchNorm.num_features,
            BatchNorm.eps,
            BatchNorm.momentum,
            BatchNorm.affine,
            BatchNorm.track_running_stats
        ]
        self.aux_bn = [module(*params) for _ in n_copies]

In [5]:
class ReLU_Block(nn.Module):
    '''
    Creates a RELU block.
    '''
    def __init__(self, in_features, out_features, dropout=True, dropout_prob=.1, batch_norm=True, bias=True):
        super(ReLU_Block, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias)
        
        if batch_norm:
            self.batch_norm = nn.BatchNorm1d(out_features)
        else:
            self.register_parameter('batch_norm', None)
            
        if dropout:
            self.dropout = nn.Dropout(dropout_prob)
        else:
            self.register_parameter('dropout', None)
        
    def forward(self, input):
        if self.dropout is not None:
            input = self.dropout(input)
        x = self.linear(input)
        if self.batch_norm is not None:
            x = self.batch_norm(x)

        return nn.functional.relu(x)
    
class SELU_Block(nn.Module):
    '''
    Creates a SELU block.
    '''
    def __init__(self, in_features, out_features, dropout=True, dropout_prob=.1, bias=True):
        super(SELU_Block, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias)
        init_SNN_weights(self.linear)
        
        if dropout:
            self.dropout = nn.AlphaDropout(dropout_prob)
        else:
            self.register_parameter('dropout', None)
        
    def forward(self, input):
        if self.dropout is not None:
            input = self.dropout(input)
        return nn.functional.selu(self.linear(input))
    
class SERLU_Block(nn.Module):
    '''
    Creates a SERLU block.
    '''
    def __init__(self, in_features, out_features, dropout=True, dropout_prob=.1, bias=True):
        super(SERLU_Block, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias)
        init_SNN_weights(self.linear)
        
        if dropout:
            self.dropout = ShiftDropout(dropout_prob)
        else:
            self.register_parameter('dropout', None)
        
    def forward(self, input):
        if self.dropout is not None:
            input = self.dropout(input)
        return SERLU(self.linear(input))

In [6]:
class classifier(nn.Module):
    def __init__(self, image_size, classes, depth, block_type, block_kws={}, name='default'):
        super(classifier, self).__init__()
        self.classes = classes
        if block_type == 'serlu':
            block = SERLU_Block
        elif block_type == 'selu':
            block = SELU_Block
        else:
            block = ReLU_Block
            
        layer_sizes = linspace(image_size, classes, depth, dtype=int)
        in_sizes, out_sizes = layer_sizes[:-1], layer_sizes[1:]
        
        net = [nn.Flatten()]
        assert depth >= 1, 'Depth must be at least 1. Recieved {0}'.format(depth)
        for in_size, out_size in zip(in_sizes[:-1], out_sizes[:-1]):
            net.append(block(in_size, out_size, **block_kws))
            
        if 'bias' in block_kws:
            bias = block_kws['bias']
        else:
            bias = True
        net.append(nn.Linear(in_sizes[-1], out_sizes[-1], bias))
        self.net = nn.Sequential(*net)
        
        self.name = name

    def forward(self, input):
        return self.net(input)
    
    def classify(self, input):
        _, classes = t.max(self.net(input), 1)
        return classes

In [7]:
def train_classifier(classifier, optimizer, epochs, device, verbose=True):
    history = []
    cum_time = 0.0
    for epoch in range(epochs):
        classifier.train()
        for batch_nb, batch in enumerate(training_loader):
            images, labels = batch[0].to(device), batch[1].to(device)

            start = time()
            #Assign nats
            x = classifier(images)

            #Compute loss
            loss = nn.functional.cross_entropy(x, labels)
            
            #Update network
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            cum_time += time()-start

        #Validate after training
        score = test_classifier(classifier, device)
        if verbose:
            print_progress({'epoch': epoch}, {'Portion correct': score})
        history.append(score)
    return history, cum_time

def test_classifier(classifier, device):
    correct = 0
    nb_samples = len(testing_set)
    with t.no_grad():
        classifier.eval()
        for batch in testing_loader:
            images, labels = batch[0].to(device), batch[1].to(device)

            #Assign classes
            assigned = classifier.classify(images)

            #Compute correct assigned
            correct += float(t.sum(assigned==labels))

    return correct/nb_samples

def attack_classifier(classifier, device):
    correct = 0
    nb_samples = len(testing_set)
    with t.no_grad():
        classifier.eval()
        for batch in testing_loader:
            images, labels = batch[0].to(device), batch[1].to(device)

            #Assign classes
            assigned = classifier.classify(images)

            #Compute correct assigned
            correct += float(t.sum(assigned==labels))

    return correct/nb_samples

In [8]:
#Choose training device (cpu vs gpu/cuda) based on availability
device = 'cuda' if t.cuda.is_available() else 'cpu'

out_size = 10
epochs = 15
depths = [4, 8, 12]

In [10]:
lr = 1e-4
relu_history = []
for depth in depths:
    name = 'relu-{0}'.format(depth)
    ReLU_net = classifier(image_size, out_size, depth, 'relu', name=name)
    ReLU_optim = t.optim.Adam(ReLU_net.parameters(), lr=lr)
    ReLU_net.to(device)
    history, cum_time = train_classifier(ReLU_net, ReLU_optim, epochs, device, verbose=False)
    relu_history.append(history)
    print('Time to train {0}: {1:.0f} m {2:.1f} s.'.format(name, cum_time//60, cum_time%60))
    print('Final validation accuracy: {0:.1%}'.format(history[-1]))

Time to train relu-4: 0 m 58.8 s.
Final validation accuracy: 57.6%
Time to train relu-8: 2 m 19.2 s.
Final validation accuracy: 57.8%
Time to train relu-12: 3 m 38.3 s.
Final validation accuracy: 57.4%


In [11]:
selu_history = []

for depth in depths:
    name = 'selu-{0}'.format(depth)
    SELU_net = classifier(image_size, out_size, depth, 'selu', {'bias': False}, name=name)
    SELU_optim = t.optim.Adadelta(SELU_net.parameters())
    SELU_net.to(device)
    history, cum_time = train_classifier(SELU_net, SELU_optim, epochs, device, verbose=False)
    selu_history.append(history)
    print('Time to train {0}: {1:.0f} m {2:.1f} s.'.format(name, cum_time//60, cum_time%60))
    print('Final validation accuracy: {0:.1%}'.format(history[-1]))

Time to train selu-4: 0 m 31.1 s.
Time to train selu-8: 1 m 10.1 s.
Time to train selu-12: 1 m 54.7 s.


In [ ]:
# SERLU_net = classifier(image_size, out_size, depth, h_size, 'serlu')
# #Move networks to selected device
# SERLU_net.to(device)

# #Set up optimizer
# SERLU_optim = t.optim.Adamax(SERLU_net.parameters(), lr=0.0005)

# #Train
# train_classifier(SERLU_net, SERLU_optim, epochs, device)

In [ ]:
def create_adversary(classifier, image, desired_label, iterations, scale=1., lr=5e-2):
    classifier.eval()
    noise = t.zeros_like(image, requires_grad=True)
    optim = t.optim.SGD([noise], lr=lr)
    for _ in range(iterations):
        optim.zero_grad()
        new_image = (image+noise)/t.max(image+noise)
        loss = nn.functional.cross_entropy(classifier(new_image), desired_label)+scale*noise.norm()
        loss.backward()
        optim.step()
    return new_image

In [ ]:
def FGSA(image, grad_data, eps):
    noise = grad_data.sign()
    new_image = t.clamp(image+eps*noise, 0, 1)
    return new_image

In [ ]:
image = image.to(device)
image.requires_grad = True
predictions = SELU_net(image)
loss = nn.functional.cross_entropy(predictions, t.tensor(label).view(1).to(device))
ReLU_net.zero_grad()
loss.backward()
new_image = FGSA(image, image.grad.data, .05)
print(nn.functional.softmax(ReLU_net(new_image), dim=1).tolist())
print(label)
plt.imshow(to_pil_image(new_image.squeeze().cpu()))

In [ ]:
def before_and_after(classifier, old_image, old_label, new_image, new_label, class_names):
    old_p = nn.functional.softmax(ReLU_net(image), 1).flatten().tolist()
    new_p = nn.functional.softmax(ReLU_net(new_image), 1).flatten().tolist()
    def _generate_text(label, p):
        return '{0}: {1:.6f}'.format(class_names[label], p[label])
    old_label = int(old_label)
    new_label = int(new_label)
    text = ['Before']
    text += [_generate_text(old_label, old_p)]
    text += [_generate_text(new_label, old_p)]
    text += ['After']
    text += [_generate_text(old_label, new_p)]
    text += [_generate_text(new_label, new_p)]
    print(*text, sep='\n')

In [ ]:
image, label = training_set[1]
plt.imshow(to_pil_image(image))

In [ ]:
image = image.to(device)
image = image.reshape((1, *image_shape))
new_label = t.tensor(1, device=device).view(1)
new_image = create_adversary(ReLU_net, image, new_label, 100, scale=1)

p_old = nn.functional.softmax(ReLU_net(image), 1).flatten().tolist()
p_new = nn.functional.softmax(ReLU_net(new_image), 1).flatten().tolist()
before_and_after(ReLU_net, image, label, new_image, new_label, training_set.classes)

new_image = new_image.squeeze()
plt.imshow(to_pil_image(new_image.cpu()))


In [ ]:
raw_images = training_set.data/255.
channel_1 = raw_images[:, :, :, 0]
channel_2 = raw_images[:, :, :, 1]
channel_3 = raw_images[:, :, :, 2]

print([channel_1.mean(), channel_2.mean(), channel_3.mean()])
print([channel_1.std(), channel_2.std(), channel_3.std()])

In [16]:
te = nn.BatchNorm1d(30)
st = te.__repr__()

In [32]:
te.track_running_stats

True